In this notebook, I run one step for TwoStreamControlModel to get intermediate outputs which I can then compare to my local run

___

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import scripts.control_utils as cu
import torch
torch.set_printoptions(linewidth=200)

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [3]:
torch.use_deterministic_algorithms(True)

In [4]:
path_to_config = 'cnxs_config/sd/sd21_encD_canny_14m.yaml'

If this results in the kernel crashing, I'm using too much GPU memory elsewhere. Shut down every other kernel and try again.

In [5]:
model = cu.create_model(path_to_config).to('cuda')

TwoStreamControlLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
[404 keys are missing from the model (hint processing and cross connections included)]
['CONTROL WEIGHTS LOADED']
Loaded model config from [cnxs_config/sd/sd21_encD_canny_14m.yaml]


In [6]:
from util import simple_describe

In [7]:
#simple_describe(model.model.diffusion_model.input_blocks, mode='cnxs', full_cls_name=True, remove='ldm.modules.')

In [8]:
#simple_describe(model.control_model.control_model.input_blocks, mode='cnxs', full_cls_name=True, remove='ldm.modules.')

In [9]:
size = 512
num_samples=1
prompt='cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed'
n_prompt='lowres, bad anatomy, worst quality, low quality'

In [10]:
def get_canny_edges():
    image_path = 'input_images/shoe.png' # chosen to fit size above
    image = cu.get_image(image_path, size=size)
    edges = cu.get_canny_edges(image, low_th=100, high_th=250)
    return edges
edges = get_canny_edges()
edges.shape

(512, 512)

In [11]:
from ldm.umer_debug_logger import udl

In [12]:
udl.set_dir('logs/cloud/', clear=True)
udl.set_condition('STEP')

In [13]:
samples, controls, latents = cu.get_sd_sample(
    guidance=edges,
    ddim_steps=50,
    num_samples=num_samples,
    model=model,
    shape=[4, size // 8, size // 8],
    control_scale=0.95,
    prompt=prompt,
    n_prompt=n_prompt,
    return_latents=True
)

Global seed set to 1999158951


[CONTROL CORRECTION OF TwoStreamControlLDM SCALED WITH 0.95]
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.5, temperature 1.0
Created random latentes of shape (1, 4, 64, 64) starting with tensor([-0.5339,  1.6289,  0.5630, -1.0880, -1.4832], device='cuda:0')
Running DDIM Sampling with 50 timesteps


DDIM Sampler:   2%|▏         | 1/50 [00:01<01:19,  1.61s/it]

α = 0.0058, α_prev = 0.0073, β = 0.9942, η = 0.5000, σ = 0.2272


DDIM Sampler:   4%|▍         | 2/50 [00:02<00:55,  1.17s/it]

...

α = 0.9609, α_prev = 0.9804, β = 0.0391, η = 0.5000, σ = 0.0499


DDIM Sampler:  98%|█████████▊| 49/50 [00:44<00:00,  1.10it/s]

α = 0.9804, α_prev = 0.9983, β = 0.0196, η = 0.5000, σ = 0.0197


DDIM Sampler: 100%|██████████| 50/50 [00:45<00:00,  1.10it/s]

α = 0.9983, α_prev = 0.9991, β = 0.0017, η = 0.5000, σ = 0.0103


In [14]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [15]:
assert False

AssertionError: 

In [ ]:
# # save initial latents
initial_rand = latents['x_inter'][0]
assert initial_rand.dim()==4 # b c w h
torch.save(initial_rand, 'latents_cloud_no_control_512.pth')
print(f'Saved noise with distribution {initial_rand.mean():.3f} ± {initial_rand.std():.3f}')

In [ ]:
assert False

In [ ]:
from PIL import Image
def show_image(im_tensor): return Image.fromarray(cu.create_image_grid(im_tensor))

In [ ]:
show_image(samples)

In [ ]:
type(model.control_model)

In [ ]:
assert False

In [ ]:
import einops
import numpy as np
from PIL import Image

def lat2img(lat, resize_to=None, output_type='pil'):
    assert lat.dim() == 4, "Expected a batch of images, not a single batch"
    
    with torch.no_grad():
        ims = model.decode_first_stage(lat)
        ims = (einops.rearrange(
            ims, 'b c h w -> b h w c'
        ) * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)
        
        if output_type == 'pil': ims = [Image.fromarray(im) for im in ims]
        
        if resize_to is not None:
            if output_type=='pil': ims = [im.resize(resize_to) for im in ims]
            else: print(f'Not resizing as output_type = {output_type} requested')
    return ims

In [ ]:
from functools import partial
from tqdm.notebook import tqdm

import torch
import einops
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw
from tqdm.notebook import tqdm
from functools import partial

def plot_latents_to_pil_grid(lats, every=1, cols=10, im_size=200, pbar=True, border=2, return_ims=True, output_type='pil'):
    if not isinstance(im_size, (list, tuple)): im_size = (im_size, im_size)
    
    lats = [lat for i, lat in enumerate(lats) if i % every == 0 or i == len(lats)-1]
    if pbar: lats = tqdm(lats)
    
    # decoce latents -> images
    ims = [lat2img(lat, resize_to=im_size, output_type=output_type)[0] for lat in lats] # removed pipe argument
    
    # add border
    ims_bordered = [ImageOps.expand(im, border=2, fill='black') for im in ims]
    im_size = (im_size[0]+border, im_size[1]+border)

    rows = len(ims) // cols
    if rows * cols < len(ims): rows += 1

    # draw background
    grid_image = Image.new('RGB', (cols * im_size[0], rows * im_size[1]), color='grey')
    draw = ImageDraw.Draw(grid_image)
    for xy in range(0,2*max(cols * im_size[0], rows * im_size[1])+1,100):
        draw.line([(xy, 0), (0, xy)], fill="white", width=1)
    
    # draw images
    for i, img in enumerate(ims_bordered):
        x_offset = (i % cols) * im_size[0]
        y_offset = (i // cols) * im_size[1]
        grid_image.paste(img, (x_offset, y_offset))

    if return_ims: return grid_image, ims
    else: return grid_image

In [ ]:
grid, ims = plot_latents_to_pil_grid(latents)

Cuda Heidelberg, control scale = 0

In [ ]:
grid